In [3]:
%%markdown
## Análise de dados

## Análise de dados


In [4]:
 # import libraries
from delta.tables import *
from pyspark.sql import SparkSession
# from pyspark import SparkConf
# from pyspark.sql import *
import pyspark
from delta import *

# Adicionado um outro caminho para o warehouse
builder = SparkSession.builder.appName("Analytic") \
.config("spark.sql.warehouse.dir", '/home/fovr/PROJETOS/olist-ml-models/src') \
.config("spark.jars.packages", "io.delta:delta-core_2.12:2.1.0") \
.config("spark.jars", "/home/fovr/PROJETOS/olist-ml-models/jdbc/sqlite-jdbc-3.34.0.jar")  \
.config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
.config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
.enableHiveSupport() 

spark = configure_spark_with_delta_pip(builder).getOrCreate()



:: loading settings :: url = jar:file:/opt/spark-3.3.2-bin-hadoop3/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fovr/.ivy2/cache
The jars for the packages stored in: /home/fovr/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2bc716d1-05b0-417e-bb1a-23811da98059;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 192ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |

23/04/08 17:32:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
%%markdown
### Silver
silver.tbAvaliacao \
silver.tbCliente \
silver.tbGeolocaliza \
silver.tbItemPedido \
silver.tbPagamentoPedido \
silver.tbPedido \
silver.tbProduto \
silver.tbVendedor 

### Silver
silver.tbAvaliacao \
silver.tbCliente \
silver.tbGeolocaliza \
silver.tbItemPedido \
silver.tbPagamentoPedido \
silver.tbPedido \
silver.tbProduto \
silver.tbVendedor 


In [19]:
%%markdown
# Issues Pagamentos

* formas de pagamentos
* Quantidade média de parcelas

# Issues Pagamentos

* formas de pagamentos
* Quantidade média de parcelas


In [6]:
spark.sql("SHOW DATABASES").show()

23/04/08 17:32:56 WARN HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
23/04/08 17:32:56 WARN HiveConf: HiveConf of name hive.stats.retries.wait does not exist
23/04/08 17:32:58 WARN ObjectStore: Version information not found in metastore. hive.metastore.schema.verification is not enabled so recording the schema version 2.3.0
23/04/08 17:32:58 WARN ObjectStore: setMetaStoreSchemaVersion called but recording version is disabled: version = 2.3.0, comment = Set by MetaStore fovr@127.0.1.1
+---------+
|namespace|
+---------+
|  default|
|   silver|
+---------+



In [7]:
spark.sql(""" SELECT * FROM silver.tbPedido""").show(5)

23/04/08 17:32:59 WARN ObjectStore: Failed to get database global_temp, returning NoSuchObjectException
23/04/08 17:33:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|            idPedido|           idCliente|descSituacao|  dtPedido|dtAprovado|   dtEnvio|dtEntregue|dtEstimativaEntrega|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|15324bb6dca4436f4...|191ba5e82034dafb0...|   delivered|2017-02-14|2017-02-14|2017-02-15|2017-02-20|         2017-03-15|
|530e59f0f148ec002...|af350704fe4d751c6...|   delivered|2017-02-14|2017-02-14|2017-02-15|2017-02-17|         2017-03-13|
|35dc5c3a87f9e9cf9...|be59c56551fe3cef6...| unavailable|2017-02-14|2017-02-14|      null|      null|         2017-03-15|
|e706d614326b76f48...|6fd87c3e71f1826c3...|   delivered|2017-02-14|2017-02-14|2017-02-17|2017-03-03|         2017-03-23|
|d2d05762a4554c9d9...|5b43ef89a22571e79...|   delivered|2017-02-14|2017-02-15|2017-02-24|2017-03-08|         2017-03-21|
+--------------------+----------

In [8]:
# Filtro para pegar apenas os ultimos 6 meses

spark.sql("""

SELECT * 
FROM silver.tbPedido
WHERE dtPedido >= add_months('2018-01-01', -6)

""").show()

+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|            idPedido|           idCliente|descSituacao|  dtPedido|dtAprovado|   dtEnvio|dtEntregue|dtEstimativaEntrega|
+--------------------+--------------------+------------+----------+----------+----------+----------+-------------------+
|d445a6e277eaa6290...|c2cd065252fa7ad04...|   delivered|2017-08-03|2017-08-03|2017-08-07|2017-08-14|         2017-09-06|
|966d505c98ea667c1...|26b32bdec947c658c...|   delivered|2017-08-03|2017-08-05|2017-08-16|2017-08-25|         2017-09-01|
|b682fc4ea655d9de9...|99c336ab501ab1c1e...|   delivered|2017-08-03|2017-08-05|2017-08-09|2017-08-28|         2017-08-23|
|52324f390c9414a6c...|5a9dae6565b54b1a3...|   delivered|2017-08-03|2017-08-04|2017-08-04|2017-08-11|         2017-09-06|
|8f2c9e626786d2b53...|98a505463599ce446...|   delivered|2017-08-03|2017-08-05|2017-08-07|2017-08-14|         2017-08-29|
|b8cd3107b41b1232e...|e7958b1dac

In [9]:


spark.sql("""

SELECT * 
FROM silver.tbPagamentoPedido

""").show(5)

+--------------------+-----------------+-----------------+----------+-----------+
|            idPedido|idPagamentoPedido|descTipoPagamento|nrParcelas|vlPagamento|
+--------------------+-----------------+-----------------+----------+-----------+
|b81ef226f3fe1789b...|                1|      credit_card|         8|      99.33|
|a9810da82917af2d9...|                1|      credit_card|         1|      24.39|
|25e8ea4e93396b6fa...|                1|      credit_card|         1|      65.71|
|ba78997921bbcdc13...|                1|      credit_card|         8|     107.78|
|42fdf880ba16b47b5...|                1|      credit_card|         2|     128.45|
+--------------------+-----------------+-----------------+----------+-----------+
only showing top 5 rows



In [90]:
# 

spark.sql("""

WITH tb1 AS (

    SELECT 
    B.*,
    A.dtPedido,
    C.idVendedor
        FROM silver.tbPedido AS A
        LEFT JOIN silver.tbPagamentoPedido as B ON A.idPedido = B.idPedido 
        LEFT JOIN silver.tbItemPedido AS C ON A.idPedido = C.idPedido
        WHERE A.dtPedido < '2018-01-01' 
        AND A.dtPedido >= add_months('2018-01-01', -6) ),
        
    tb2 AS (
        
        SELECT idVendedor,
            descTipoPagamento,
            COUNT(DISTINCT idPedido) AS qtPedidoMeioPagamento,
            SUM(vlPagamento) AS vlPedidoPagamento

        FROM tb1

        GROUP BY idVendedor, descTipoPagamento
        ORDER BY idVendedor, descTipoPagamento )
        
    SELECT  idVendedor 
           -- , SUM(CASE WHEN descTipoPagamento = 'boleto' THEN vlPedidoPagamento ELSE 0 END) AS vl_boleto_pedido           
           -- , SUM(CASE WHEN descTipoPagamento = 'credit_card' THEN vlPedidoPagamento ELSE 0 END) AS vl_credit_card_pedido      
           -- , SUM(CASE WHEN descTipoPagamento = 'voucher' THEN vlPedidoPagamento ELSE 0 END) AS vl_voucher_pedido           
           -- , SUM(CASE WHEN descTipoPagamento = 'debit_card' THEN vlPedidoPagamento ELSE 0 END) AS vl_debit_card_pedido
            
            , SUM(CASE WHEN descTipoPagamento = 'boleto' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_boleto_pedido           
            , SUM(CASE WHEN descTipoPagamento = 'credit_card' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_credit_card_pedido      
            , SUM(CASE WHEN descTipoPagamento = 'voucher' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_voucher_pedido           
            , SUM(CASE WHEN descTipoPagamento = 'debit_card' THEN qtPedidoMeioPagamento ELSE 0 END)/ SUM(qtPedidoMeioPagamento) AS pct_qt_debit_card_pedido
            
            FROM   tb2
    GROUP BY idVendedor

""").show(truncate=False)



+--------------------------------+--------------------+-------------------------+---------------------+------------------------+
|idVendedor                      |pct_qt_boleto_pedido|pct_qt_credit_card_pedido|pct_qt_voucher_pedido|pct_qt_debit_card_pedido|
+--------------------------------+--------------------+-------------------------+---------------------+------------------------+
|b3f19518fcec265b2e97af287725f981|0.5                 |0.5                      |0.0                  |0.0                     |
|062ce95fa2ad4dfaedfc79260130565f|0.16666666666666666 |0.8333333333333334       |0.0                  |0.0                     |
|791cfcfe22fe4a771ece27f90017da92|0.0                 |1.0                      |0.0                  |0.0                     |
|2009a095de2a2a41626f6c6d7722678d|0.4                 |0.6                      |0.0                  |0.0                     |
|8e6cc767478edae941d9bd9eb778d77a|0.20754716981132076 |0.6981132075471698       |0.09433962264150

In [75]:
spark.sql(""" SELECT 
    B.*,
    A.dtPedido
    --C.idVendedor
        FROM silver.tbPedido AS A
        LEFT JOIN silver.tbPagamentoPedido as B ON A.idPedido = B.idPedido 
        LEFT JOIN silver.tbItemPedido AS C ON A.idPedido = C.idPedido
        WHERE  C.idVendedor IS NOT NULL """).show(truncate=False)

+--------------------------------+-----------------+-----------------+----------+-----------+----------+
|idPedido                        |idPagamentoPedido|descTipoPagamento|nrParcelas|vlPagamento|dtPedido  |
+--------------------------------+-----------------+-----------------+----------+-----------+----------+
|6ea2f835b4556291ffdc53fa0b3b95e8|1                |credit_card      |10        |356.12     |2017-11-24|
|b01875821b8dcb6abc61776f0f971bce|1                |credit_card      |10        |142.16     |2017-11-24|
|b01875821b8dcb6abc61776f0f971bce|1                |credit_card      |10        |142.16     |2017-11-24|
|c263211bd219538f7c031591e87ef0d7|1                |boleto           |1         |753.94     |2017-11-24|
|c263211bd219538f7c031591e87ef0d7|1                |boleto           |1         |753.94     |2017-11-24|
|fb9b2930f4a30f96c7cc98eaaa85e5f0|1                |credit_card      |5         |84.43      |2017-11-24|
|52e5445828db681d83dccb33a15ecac4|1                |cre

In [76]:
spark.sql(""" SELECT * 
        FROm silver.tbItemPedido AS C 
        WHERE C.idpedido = '6ea2f835b4556291ffdc53fa0b3b95e8' """).show()

+--------------------+------------+--------------------+--------------------+-------------+-------+-------+
|            idPedido|idPedidoItem|           idProduto|          idVendedor|dtLimiteEnvio|vlPreco|vlFrete|
+--------------------+------------+--------------------+--------------------+-------------+-------+-------+
|6ea2f835b4556291f...|           1|be021417a6acb56b9...|f5f46307a4d15880c...|   2017-11-30|  339.0|  17.12|
+--------------------+------------+--------------------+--------------------+-------------+-------+-------+

